In [1]:
print("Hello, World!")

Hello, World!


In [10]:
!pip3 install openai python-dotenv pypdf


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip3 freeze > requirements.txt

In [12]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader

In [13]:
load_dotenv(override=True)

gemini_api_key = os.getenv("GOOGLE_API_KEY")

if not gemini_api_key:
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [15]:
def read_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        return None

In [20]:
dietary_guidelines_text = read_pdf("resources/dietary_guidelines.pdf")

if not dietary_guidelines_text:
    raise ValueError("Failed to read the dietary guidelines PDF file.")

In [21]:
dietary_guidelines_text[:10]

'Dietary\nGu'